In [1]:
import pandas as pd
import numpy as np

- `pd.merge()` merges two dataframes
    - by default, an inner join is performed, which only contains keys present in both dataframes
    - we can specify other merges using the `how` parameter
    - the columns on which to merge can also be specified, as well as indices
    - each dataframe also has a `.join()` method which works the same
- `pd.concat()` stacks dataframes and series on top of each other
    - we can also specify to stack along the columns
- `a.combine_first(b)` is the perfect method for combining dataframes with overlap in values
    - by default, na values in `a` are overwritten by values in `b`
    - but if a values is present in `a`, it is being preferred over any values in `b`
- `df.stack()` rotates columns to rows, i.e. the lowest level columns are turned into a row index
- `df.unstack()` does the opposite: the lowest level row index is turned into columns
- both functions have parameters to control which level to turn into rows or columns
- `df.pivot()` is a convenient method to pivot tables, it takes in several parameters:
    - `index` defines which column from the dataframe to turn into the index
    - `columns` defines which column to turn into new columns
    - `values` defines which column to take values from
- `df.melt()` is the opposite: it takes in some column and uses it as a new row index

# 8.1 Hierarchical Indexing

Allows you to have multiple index levels on an axis.

In [2]:
# by using a 2d list as an input to the index parameter, a hierarchical index is constructed automatically
data = pd.Series(
    np.random.uniform(size=9),
    index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.131209
   2    0.621316
   3    0.701095
b  1    0.606910
   3    0.242248
c  1    0.901759
   2    0.062977
d  2    0.561175
   3    0.743152
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# can select subsets of the data through only one level of the index
data["b"]

1    0.606910
3    0.242248
dtype: float64

In [5]:
data["b":"c"]

b  1    0.606910
   3    0.242248
c  1    0.901759
   2    0.062977
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1    0.606910
   3    0.242248
d  2    0.561175
   3    0.743152
dtype: float64

In [7]:
# multiple level indexing
data.loc[:, 2]  # pick all highest-level indices and index "2" from the second-highest level

a    0.621316
c    0.062977
d    0.561175
dtype: float64

In [8]:
# rearrange into dataframe
# the lowest level index becomes the new columns
data.unstack()

,1,2,3
a,0.131209,0.621316,0.701095
b,0.606910,NaN,0.242248
c,0.901759,0.062977,NaN
d,NaN,0.561175,0.743152


In [9]:
# rearrange back into series
data.unstack().stack()

a  1    0.131209
   2    0.621316
   3    0.701095
b  1    0.606910
   3    0.242248
c  1    0.901759
   2    0.062977
d  2    0.561175
   3    0.743152
dtype: float64

In [21]:
# on dataframes, both axes can have a hierarchical index
# we again use a 2d list to create the hierarchical index
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[["gdp", "gdp", "pop", "pop"], [2019, 2020, 2019, 2020]], columns=[["Ohio", "Ohio", "Colorado"], ["West", "East", "West"]])
frame

Ohio      Colorado
         West East     West
gdp 2019    0    1        2
    2020    3    4        5
pop 2019    6    7        8
    2020    9   10       11

In [23]:
# the indices can have names too
frame.index.names = ["val", "year"]
frame.columns.names = ["state", "region"]
frame

state    Ohio      Colorado
region   West East     West
val year                   
gdp 2019    0    1        2
    2020    3    4        5
pop 2019    6    7        8
    2020    9   10       11

In [24]:
# check level of index
frame.columns.nlevels

2

In [25]:
# you can now pick partial groups
frame["Ohio"]

region    West  East
val year            
gdp 2019     0     1
    2020     3     4
pop 2019     6     7
    2020     9    10

In [14]:
# finally, the multiindex can be created by itself
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]], names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting Levels

In [26]:
# can easily swap two levels
frame.swaplevel("val", "year")

state    Ohio      Colorado
region   West East     West
year val                   
2019 gdp    0    1        2
2020 gdp    3    4        5
2019 pop    6    7        8
2020 pop    9   10       11

In [27]:
# sort_index by default uses all index levels
# with the sorting starting with the highest level index
frame.sort_index()

state    Ohio      Colorado
region   West East     West
val year                   
gdp 2019    0    1        2
    2020    3    4        5
pop 2019    6    7        8
    2020    9   10       11

In [28]:
# but the level can be chosen explicitly
frame.sort_index(level=1)

state    Ohio      Colorado
region   West East     West
val year                   
gdp 2019    0    1        2
pop 2019    6    7        8
gdp 2020    3    4        5
pop 2020    9   10       11

In [29]:
# performance is best if the primary index to sort is at level 0
frame.swaplevel(0, 1).sort_index(level=0)

state    Ohio      Colorado
region   West East     West
year val                   
2019 gdp    0    1        2
     pop    6    7        8
2020 gdp    3    4        5
     pop    9   10       11

## Summary Statistics by Level

In [32]:
# summary statistics can be computed based on the level
# e.g. when using mean(), for each specified index, we compute the mean on each column
# which reduces the index of the dataframe by one level
frame.groupby(level="val").mean()

state  Ohio      Colorado
region West East     West
val                      
gdp     1.5  2.5      3.5
pop     7.5  8.5      9.5

In [34]:
frame.T.groupby(level="state").sum()

val       gdp       pop     
year     2019 2020 2019 2020
state                       
Colorado    2    5    8   11
Ohio        1    7   13   19

## Indexing with a DataFrame's columns

In [35]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1), "c": ["one", "one", "one", "two", "two", "two", "two"], "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [36]:
# set_index creates a new dataframe using the specified columns as indices
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [37]:
# the columns are removed but can be kept using drop=False
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [38]:
# using reset_index(), the levels can be removed again
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

There are several ways to combine and merge datasets:

- `pandas.merge`: Connect rows based on one or more keys, similar to join from relational databases.
- `pandas.concat`: Concatenate or "stack" objects together along an axis.
- `combine_first`: Splice together overlapping data to fill in missing values in one object with values from another.

## Database-Style DataFrame Joins

In [39]:
# merge or join operations combine datasets by linking rows using one or more keys
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": pd.Series(range(3), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [40]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [42]:
# this is a many-to-one join, each equal key in df2 gets assigned to each one in df1
# note how no merging column was specified: pandas chose which one to use by comparing the names

# by default, pandas does an inner join, meaning that only key combinations that exist in both
# dataframes will be part of the result
# the resulting dataframe contains one row per key-pair

# Given two dataframes, merge will unify two dataframes into a single one based on a key column.
# If the key column is not specified, pandas can choose, if two columns in the dfs have the same name.
# By default, the outcome will be an inner join, which means:
# - the resulting dataframe will contain one row per possible row combination from the initial dataframes
# - any rows that contain a key that is only present in one dataframe get ignored
# - so e.g. if we have key "a" in df1 3 times and in df2 2 times, we will have 6 possible combinations, i.e.
#   6 rows in the new df
# - the columns will be the unification of the columns from the initial dataframes
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [45]:
# it is good practice to specify the key column to merge on, using the "on" parameter
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [46]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"], "data2": pd.Series(range(3), dtype="Int64")})
print(df3.columns, df4.columns)

Index(['lkey', 'data1'], dtype='object') Index(['rkey', 'data2'], dtype='object')


In [47]:
# for each dataframe, a different key column can be specified using left_on and right_on
# furthermore, by default an "inner" join is done, so only intersecting keys are kept
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [49]:
# Ee can override the default inner join using the "how" parameter:
# e.g. outer combines left and right join, meaning that each key
# from both dataframes will be present at least once
# if some key is only present in one dataframe, the resulting df
# will have a na value in the respective column
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [41]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [50]:
# left means all keys from the left table will be present in the resulting table
# any entries from the right table with a key not present in the left one will be ignored
pd.merge(df1, df2, how="left")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


In [51]:
# right is similar, though it only uses all keys from right table
# note how swapping the dataframes and using right instead of left is the same result
pd.merge(df2, df1, how="right")

,key,data2,data1
0,b,1,0
1,b,1,1
2,a,0,2
3,c,<NA>,3
4,a,0,4
5,a,0,5
6,b,1,6


In [45]:
pd.merge(df1, df2, how="inner") # as before: the default

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [52]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"], "data2": pd.Series(range(5), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [47]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [53]:
# many-to-many merges are possible as well
# which happen if both dfs contain a key several times
# in this case, the cartesian product is created
# since there were 2 rows with "b" in the left df and 3 in the right
# the result now has 6
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [54]:
# you can also merge with multiple keys
left = pd.DataFrame({"key1": ["foo", "foo", "bar"], "key2": ["one", "two", "one"], "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"], "key2": ["one", "one", "one", "two"], "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [55]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [52]:
# simply by using a list of key identifiers rather than a single string, multiple
# keys are used for merging
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [56]:
# overlapping column names get renamed
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [57]:
# you can also specify your own suffixes
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


- `merge` unifies two dataframes into one using keys to identify which rows to merge
- by default an `inner` join is created, which means that only overlapping keys will remain in the result
- using the `how` parameter, `left`, `right` and `outer` joins can be created as well
- the `on` parameter can be used to select which columns to use as a key
- `left_on` and `right_on` can be used to pick different keys per dataframe
- by passing a list to `on`, `left_on` and `right_on`, multiple columns can be used as keys
- overlapping columns will be renamed, custom renames can be created using the `suffixes` parameter

## Merging on Index

In [65]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"], "value": pd.Series(range(6), dtype="Int64")})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [66]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
right1

,group_val
a,3.5
b,7.0


In [67]:
# using left_on, left_index, right_on, right_index, different keys can be selected per dataframe
# in this case, the "key" column is used from the left df
# and the index is used for the right df
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [68]:
lefth = pd.DataFrame({
    "key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
    "key2": [2000, 2001, 2002, 2001, 2002],
    "data": pd.Series(range(5), dtype="Int64")}
)
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [69]:
righth_index = pd.MultiIndex.from_arrays([
    ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
    [2001, 2000, 2000, 2000, 2001, 2002]
])
righth = pd.DataFrame({
    "event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64", index=righth_index),
    "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64", index=righth_index)
})
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [60]:
# on hierarchical indexed data, joining on index means joining on multiple keys
# so, you have to specify multiple keys on the other df
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [61]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [63]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=["a", "c", "e"], columns=["Ohio", "Nevada"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [64]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=["b", "c", "d", "e"], columns=["Missouri", "Alabama"]).astype("Int64")
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [64]:
# can also use index from both dfs
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [66]:
# the dataframes also have a method join, that can produce the same result
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [67]:
# join by default uses the index as keys and produces a left join
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [69]:
# but a key column can be specified too
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [70]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], index=["a", "c", "e", "f"], columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [72]:
# you can also pass in multiple dataframes, leading to a simple index based merge
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [73]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


- using `right_index=True` and `left_index=True`, the index can be used as the key column
- if the index used as keys from one dataframe is a multi-index, the number of keys from the other df must be the same as the level of the index
- `pandas.DataFrame` also has a `join()` method, which produces the same output as merge, but uses a left join on the index as default
- `join()` can also take in multiple dataframes as a list, which will be merged into the left df

## Concatenating Along an Axis

In [2]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [4]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [5]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s1

a    0
b    1
dtype: Int64

In [6]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s2

c    2
d    3
e    4
dtype: Int64

In [7]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")
s3

f    5
g    6
dtype: Int64

In [8]:
# by default, pandas concatenates along the index
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [9]:
# passing in axis="columns", a dataframe would be constructed
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [11]:
# you can pass in a join operator as well
# using "inner" join, only intersecting keys will be kept
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [12]:
# we can easily create a hierarchical index, to identify from which series elements came from
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [13]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [14]:
# if we further combine along the columns, the keys become the DF column headers
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [70]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"], columns=["one", "two"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [71]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"], columns=["three", "four"])
df2

,three,four
a,5,6
c,7,8


In [72]:
pd.concat([df1, df2])

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [18]:
# this naturally extends to dataframes, where the keys are used as a hierarchical index on the columns
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [19]:
# the same thing can be applied when passing in a dictionary of objects
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [20]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), columns=["a", "b", "c", "d"])
df1

,a,b,c,d
0,0.411339,-0.942263,-0.505416,1.267344
1,0.479324,1.295759,-0.767059,-0.705542
2,0.544238,-0.271915,0.324274,-0.694395


In [21]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)), columns=["b", "d", "a"])
df2

,b,d,a
0,0.930768,2.118428,1.458909
1,1.384818,0.284277,1.705873


In [22]:
# finally, if we want to ignore an index, we can do so:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.411339,-0.942263,-0.505416,1.267344
1,0.479324,1.295759,-0.767059,-0.705542
2,0.544238,-0.271915,0.324274,-0.694395
3,1.458909,0.930768,NaN,2.118428
4,1.705873,1.384818,NaN,0.284277


- `pd.concat` on a series concatenates the input series into a single unified one along the index by default
- by passing in `axis="columns"`, a dataframe will be constructed where each series becomes one column in the new df
- the index is preserved, any index not present in one series will simply have a na value in the resulting df
- by passing in `join="inner"`, only rows will be created from keys that are present in all dfs
- using the `keys` parameter, we can add identifiers to the initial series objects, creating a hierarchical index in the result
- by default, on dataframes `pd.concat` behaves similar: new rows will be created with each column not present in the initial dfs having a NA value
- by using `axis="columns"`, indices will be matched and the columns are concatenated
- here, we can also specify `keys` again
- the same result can be achieved by passing in a dictionaries, where the keys will become the new index names
- `ignore_index=True` let's us reset the index

## Combining Data with Overlap

In [23]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [24]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.], index=["a", "b", "c", "d", "e", "f"])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [25]:
# if "a" is na at a certain position, get b, else get a
# however, mind that index labels are ignored!
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [26]:
# pandas has a special function for this:
# it behaves the same as where, but aligns on the index
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [74]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan], "b": [np.nan, 2., np.nan, 6.], "c": range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [75]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.], "b": [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [76]:
# with dataframes, the same thing is done column-by-column
# any columns/indexes present in only one dataframe will be propagated to the result
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


- `combine_first` updates a series by filling any na values using the values of another series based on the index
- on dataframes, the behavior remains the same, but the updates are done on each column that is named the same

# 8.3 Reshaping and Pivoting

## Reshaping with Hierarchical Indexing

- `stack`: Rotates from columns to rows
- `unstack`: Rotates from rows to columns

In [78]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(["Ohio", "Colorado"], name="state"), columns=pd.Index(["one", "two", "three"], name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
# stack: columns become rows
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [82]:
result.reset_index()

,state,number,0
0,Ohio,one,0
1,Ohio,two,1
2,Ohio,three,2
3,Colorado,one,3
4,Colorado,two,4
5,Colorado,three,5


In [34]:
# unstack: rows become columns
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [35]:
# by default, stack and unstack work on the innermost level
# you can specify along which level to stack, though
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [36]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [37]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [38]:
# stacking may lead to missing data
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [39]:
# which behaves this way, because inverting the operation is easier this way
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [42]:
data2.unstack().stack(future_stack=True)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [43]:
df = pd.DataFrame({"left": result, "right": result + 5}, columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [44]:
# when unstacking, the unstacked level becomes the lowest level in the result
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [46]:
# we can indicate the level to stack as well
# again this becomes the lowest level
df.unstack(level="state").stack(level="side", future_stack=True)

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

- `stack` pivots columns into rows: for each cell in the dataframe, one row will be created in the result
- the result will be a series with a hierarchical index, where the initial index will be on the highest level and the previous columns will be the lowest index
- `unstack` pivots back rows into columns: with a hierarchical indexed dataframe, the lowest index will become the new columns
- this default behavior can be overriden: e.g. by specifying `level=0`, the highest index will be used
- or by using `level="name"` the name can be used as an identifier
- by using the `keys` parameter, we can create a hierarchical index in the result, which lets us identify from which series the values came from

## Pivoting "Long" to "Wide" Format

In [62]:
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]] # pick certain columns, all rows
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [63]:
periods = pd.PeriodIndex.from_fields(year=data.pop("year"), quarter=data.pop("quarter"))    # pop returns a column while deleting it
periods.name = "date"
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [64]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [65]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [66]:
long_data = (data.stack().reset_index().rename(columns={0: "value"}))
long_data.head()

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


This format is called long format, where each row is one observation.

This is often used in SQL databases as a fixed scheme, as the number of distinct values in the
item column can change dynamically without having to update the table scheme. `date` and `item`
are usually used as the primary key in this format.

In [68]:
# pivot gets back what we had before
# index parameter defines what should be used as index
# columns specifies from which column the newly created columns will come from
pivoted = long_data.pivot(index="date", columns="item", values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [75]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data.head()

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.435294
1,1959-01-01,infl,0.000,0.326041
2,1959-01-01,unemp,5.800,0.749541
3,1959-04-01,realgdp,2778.801,0.515242
4,1959-04-01,infl,2.340,1.064019


In [76]:
# you can also easily pivot with two values
# which creates a hierarchical index
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.326041 -0.435294  0.749541
1959-04-01  2.34  2778.801   5.1  1.064019  0.515242 -1.200307
1959-07-01  2.74  2775.488   5.3  0.483256  1.572445  1.291422
1959-10-01  0.27  2785.204   5.6 -0.150023 -1.501374 -0.187261
1960-01-01  2.31  2847.699   5.2  0.458352  1.705769  1.994697

In [78]:
# we can have the same outcome by creating a hierarchical index and calling unstack
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.326041 -0.435294  0.749541
1959-04-01  2.34  2778.801   5.1  1.064019  0.515242 -1.200307
1959-07-01  2.74  2775.488   5.3  0.483256  1.572445  1.291422
1959-10-01  0.27  2785.204   5.6 -0.150023 -1.501374 -0.187261
1960-01-01  2.31  2847.699   5.2  0.458352  1.705769  1.994697

- `pivot` lets us reshape a dataframe from long to wide format
- in the long format, each data value in a database is saved in its own row instead of in a column/row combination
- the pivot function lets us pick the index to use using the `index` parameter
- the `columns` parameter decides which column in the initial df will be used to create the new columns: each unique value in the specified columns will become a new column index
- similarly, `index` let's us pick which column will become the index, where each unique value in this column will be one index
- finally, `values` lets us pick the column from which the data will come from
- if we do not pick the values and the initial df had multiple columns, a hierarchical index will be created

## Pivoting "Wide" to "Long" Format

In [79]:
# pd.melt is the inverse operation to pivot
# it merges multiple columns into a single one
df = pd.DataFrame({"key": ["foo", "bar", "baz"], "A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [80]:
# id_vars specifies which columns to use as group indicators
# it may be e.g. a group indicator and the other columns are data values
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [81]:
# pivot can reshape back:
reshaped = melted.pivot(index="key", columns="variable", values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [82]:
# the pivoting creates an index from the key column
# use reset_index to move the data back to a column
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [83]:
# you can also specify a subset of columns to use
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [84]:
# group identifiers can be left out too:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [85]:
pd.melt(df, value_vars=["A", "B", "key"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,key,foo
7,key,bar
8,key,baz


- `melt` is the opposite of `pivot`: it transforms several columns into one
- the `id_vars` parameter can be used to pick some key, but it can be left out
- `value_vars` lets us pick which columns to use in the new df, for each entry in these columns one row will be added containing with one column specifying where
from which column the value came from and another one containing the actual value